In [1]:
import LinearResponseVariationalBayes as vb
import LinearResponseVariationalBayes.SparseObjectives as obj_lib
import LinearResponseVariationalBayes.OptimizationUtils as opt_lib
import autograd.numpy as np

In [2]:
class dummy_np_test(object):
    def __init__(sefl):
        pass
    def assert_array_almost_equal(self, x, y):
        assert np.linalg.norm(x - y) < 1e-6

np_test = dummy_np_test()

In [3]:
class QuadraticModel(object):
    def __init__(self, dim):
        self.dim = dim
        self.param = vb.VectorParam('theta', size=dim)

        vec = np.linspace(0.1, 0.3, num=dim)
        self.matrix = np.outer(vec, vec) + np.eye(dim)
        self.vec = vec

        self.objective = obj_lib.Objective(self.param, self.get_objective)

    def get_objective(self):
        theta = self.param.get()
        objective = 0.5 * theta.T @ self.matrix @ theta + self.vec @ theta
        return objective

    # Testing functions that use the fact that the optimum has a closed form.
    def get_true_optimum_theta(self):
        theta = -1 * np.linalg.solve(self.matrix, self.vec)
        return theta

    def get_true_optimum(self):
        # ...in the free parameterization.
        theta = self.get_true_optimum_theta()
        self.param.set_vector(theta)
        return self.param.get_free()
    
model = QuadraticModel(3)

init_x = np.zeros(model.dim)

In [4]:
opt_x, opt_result = opt_lib.minimize_objective_bfgs(model.objective, init_x, precondition=False)
np_test.assert_array_almost_equal(model.get_true_optimum(), opt_result.x)
np_test.assert_array_almost_equal(model.get_true_optimum(), opt_x)

opt_x, opt_result = opt_lib.minimize_objective_trust_ncg(model.objective, init_x, precondition=False)
np_test.assert_array_almost_equal(model.get_true_optimum(), opt_result.x)
np_test.assert_array_almost_equal(model.get_true_optimum(), opt_x)


Iter  0  value:  0.0
Optimization terminated successfully.
         Current function value: -0.061404
         Iterations: 2
         Function evaluations: 3
         Gradient evaluations: 3
Iter  0  value:  0.0
Iter  1  value:  -0.06140350877192982
Optimization terminated successfully.
         Current function value: -0.061404
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2
         Hessian evaluations: 0


In [5]:
hessian, inv_hess_sqrt, hessian_corrected = \
    opt_lib.set_objective_preconditioner(model.objective, init_x)
np_test.assert_array_almost_equal(model.matrix, hessian)

In [6]:
opt_x, opt_result = opt_lib.minimize_objective_bfgs(model.objective, init_x, precondition=True)
np_test.assert_array_almost_equal(model.get_true_optimum(), opt_x)

opt_x, opt_result = opt_lib.minimize_objective_trust_ncg(model.objective, init_x, precondition=True)
np_test.assert_array_almost_equal(model.get_true_optimum(), opt_x)


Iter  0  value:  0.0
Optimization terminated successfully.
         Current function value: -0.061404
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2
Iter  0  value:  0.0
Iter  1  value:  -0.061403508771929835
Optimization terminated successfully.
         Current function value: -0.061404
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2
         Hessian evaluations: 0


In [22]:
def initial_optimization(x):
    new_x = np.random.random(len(x))
    return new_x, new_x

def take_gradient_step(x):
    grad = model.objective.fun_free_grad(x)
    return x - 0.5 * grad, x

opt_x, converged, x_conv, f_conv, grad_conv, obj_opt, opt_results = \
    opt_lib.repeatedly_optimize(
        model.objective,
        take_gradient_step,
        init_x,
        initial_optimization_fun=initial_optimization,
        keep_intermediate_optimizations=True,
        max_iter=100)
    
print(np.linalg.norm(opt_x - model.get_true_optimum()))
np_test.assert_array_almost_equal(model.get_true_optimum(), opt_x)

5.7604516071836537e-05


AssertionError: 